1. To create C code snippet and integrate them into Python code, first create either:
- a C code file: my_code.c
- a C++ code file: my_code.cpp

2. Compile C code into a Python extension using setuptools


In [ ]:
from setuptools import setup, Extension

setup(
    name='my_c_module',
    version='1.0',
    ext_modules=[Extension('my_c_module', sources=['my_code.c'])],
)


Then run :
```bash
python setup.py build_ext --inplace
```

The use the C extension in your Python code  



In [ ]:
import my_c_module

my_c_module.greet("World")

## Example of C code into Python for sorting array

1. Create a C file (insertion_sort.c)
2. Create a setup.py file
3. Build the C extension with command: **python setup.py build_ext --inplace**
4. Create your Python file (eg use_insertion_sort.py)
5. Run it: **python use_insertion_sort.py**

In [ ]:
#include <Python.h>

static PyObject* py_insertion_sort(PyObject* self, PyObject* args) {
    PyObject* list_obj;
    if (!PyArg_ParseTuple(args, "O", &list_obj)) {
        return NULL;
    }

    if (!PyList_Check(list_obj)) {
        PyErr_SetString(PyExc_TypeError, "Expected a list");
        return NULL;
    }

    Py_ssize_t size = PyList_Size(list_obj);

    for (Py_ssize_t i = 1; i < size; i++) {
        PyObject* key = PyList_GetItem(list_obj, i);
        Py_ssize_t j = i - 1;

        while (j >= 0 && PyObject_RichCompareBool(key, PyList_GetItem(list_obj, j), Py_LT)) {
            PyList_SetItem(list_obj, j + 1, PyList_GetItem(list_obj, j));
            j = j - 1;
        }

        PyList_SetItem(list_obj, j + 1, key);
    }

    return list_obj;
}

static PyMethodDef InsertionSortMethods[] = {
    {"insertion_sort", py_insertion_sort, METH_VARARGS, "Sort a list using insertion sort."},
    {NULL, NULL, 0, NULL}
};

static struct PyModuleDef insertion_sort_module = {
    PyModuleDef_HEAD_INIT,
    "insertion_sort",   /* name of module */
    NULL,          /* module documentation, may be NULL */
    -1,           /* size of per-interpreter state of the module,
                      or -1 if the module keeps state in global variables */
    InsertionSortMethods
};

PyMODINIT_FUNC PyInit_insertion_sort(void) {
    return PyModule_Create(&insertion_sort_module);
}

In [ ]:
# setup.py file
from setuptools import setup, Extension

setup(
    name='insertion_sort_c',
    version='1.0',
    ext_modules=[Extension('insertion_sort_c', sources=['insertion_sort.c'])],
)

In [ ]:
# use_insertion_sort.py file
from insertion_sort_c import insertion_sort

my_list = [5, 2, 9, 1, 5, 6]
sorted_list = insertion_sort(my_list)

print("Original list:", my_list)
print("Sorted list:", sorted_list)

### To do same stuff with C++, use Pybind11 library

In [ ]:
#include <pybind11/pybind11.h>

namespace py = pybind11;

int add(int a, int b) {
    return a + b;
}

PYBIND11_MODULE(my_cpp_module, m) {
    m.doc() = "pybind11 example plugin"; // optional module docstring

    m.def("add", &add, "A function that adds two numbers"); 
}

In [ ]:
# setup.py file
from setuptools import setup, Extension
from pybind11.setup_helpers import Pybind11Extension

ext_modules = [
    Pybind11Extension(
        "my_cpp_module",
        ["my_cpp_module.cpp"],
        # ... other arguments
    ),
]

setup(
    name="my_cpp_module",
    ext_modules=ext_modules,
    # ... other setup arguments
)

Compile and Install: python setup.py build_ext --inplace

In [ ]:
# use in your Python code
import my_cpp_module

result = my_cpp_module.add(2, 3)
print(result)  # Output: 5